In [ ]:
from google.colab import drive          # connect to drvie
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# install needed libraries
!pip install zstd
!pip install zstandard
# For TensorBoard
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

     |████████████████████████████████| 471kB 3.2MB/s 
  Created wheel for zstd: filename=zstd-1.4.4.0-cp36-cp36m-linux_x86_64.whl size=1129664 sha256=e6c1174985ab4aa12f986c25985ec8b04146d8bcd7ddf77f45a87141ba256f02
  Stored in directory: /root/.cache/pip/wheels/ce/0d/5a/3c2ccf5522d1cd3ecdeb2af8c9889179d7a204376b1d5c042d
Successfully built zstd
     |████████████████████████████████| 3.5MB 3.3MB/s 
--2020-04-20 13:29:06--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.197.27.35, 34.232.135.84, 34.233.35.85, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.197.27.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  37.3MB/s    in 0.4s    

2020-04-20 13:29:07 (37.3 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-l

In [ ]:
# imports
import tensorflow
from tensorflow import keras
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard
from sys import exit
import zstd
import zstandard

Using TensorFlow backend.


In [ ]:
LogDir = './log'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LogDir)
)

get_ipython().system_raw('./ngrok http 6006 &')

! curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://7bb8985c.ngrok.io


In [ ]:
FileAddress = r"/content/drive/My Drive/ML/Python Plays NFS/split Data(python plays NFS)/{}"
OutPutName = ["x_train.npy", "y_train.npy", "x_test.npy", "y_test.npy"]

In [ ]:
def decompress_zstd(input_file, output_file):                      
    with open(input_file, 'rb') as compressed:
        decomp = zstandard.ZstdDecompressor()

        with open(output_file, 'wb') as destination:
            decomp.copy_stream(compressed, destination)

In [ ]:
print("Decompressing!")
decompress_zstd(FileAddress.format("x_train.npy.zst"), OutPutName[0])
decompress_zstd(FileAddress.format("y_train.npy.zst"), OutPutName[1])
decompress_zstd(FileAddress.format("x_test.npy.zst"), OutPutName[2])
decompress_zstd(FileAddress.format("y_test.npy.zst"), OutPutName[3])
print("Done!")

Decompressing!
Done!


In [ ]:
# Load train data into ram
x_train = np.load("x_train.npy", allow_pickle=True)
y_train = np.load("y_train.npy", allow_pickle=True)

In [ ]:
x_train = x_train.reshape(x_train.shape[0], 224, 224, 1)            # reshaping data

In [ ]:
print("Data Is Ready For training!\n")

Data Is Ready For training!



In [ ]:
DataTestSize = 0.07
Epochs = 32
batch_size = 32

In [ ]:
model = Sequential()

# 1st Convolutional Layer
model.add(Conv2D(filters=96, input_shape=(224, 224, 1), kernel_size=(11, 11), strides=(4, 4), padding='valid'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))
# Batch Normalisation before passing it to the next layer
model.add(BatchNormalization())

# 2nd Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(11, 11), strides=(1, 1), padding='valid'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))
# Batch Normalisation
model.add(BatchNormalization())

# 3rd Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), padding='valid'))
model.add(Activation('relu'))
# Batch Normalisation
model.add(BatchNormalization())

# 4th Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), padding='valid'))
model.add(Activation('relu'))
# Batch Normalisation
model.add(BatchNormalization())

# 5th Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), padding='valid'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))
# Batch Normalisation
model.add(BatchNormalization())

# Passing it to a dense layer
model.add(Flatten())
# 1st Dense Layer
model.add(Dense(4096, input_shape=(224*224*3,)))
model.add(Activation('relu'))
# Add Dropout to prevent overfitting
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# 2nd Dense Layer
model.add(Dense(4096))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# 3rd Dense Layer
model.add(Dense(1000))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# Output Layer
model.add(Dense(3))
model.add(Activation('softmax'))

model.summary()

# (4) Compile
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 54, 54, 96)        11712     
_________________________________________________________________
activation_1 (Activation)    (None, 54, 54, 96)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 27, 27, 96)        384       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 17, 17, 256)       2973952   
_________________________________________________________________
activation_2 (Activation)    (None, 17, 17, 256)       0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 256)        

In [ ]:
tbCallBack = TensorBoard(log_dir=LogDir, histogram_freq=1,                  # TensorBoard
                         write_graph=True,
                         write_images=True)

In [ ]:
print("Training...")
model.fit(x_train, y_train, batch_size=batch_size, epochs=Epochs, verbose=1, validation_split=0.08, callbacks=[tbCallBack])
print("Done training")

Training...
Train on 47494 samples, validate on 4130 samples
Epoch 1/32
47494/47494 [==============================] - 234s 5ms/step - loss: 1.0851 - accuracy: 0.4916 - val_loss: 1.4406 - val_accuracy: 0.4642
Epoch 2/32
47494/47494 [==============================] - 225s 5ms/step - loss: 0.7736 - accuracy: 0.6606 - val_loss: 0.8049 - val_accuracy: 0.6557
Epoch 3/32
47494/47494 [==============================] - 222s 5ms/step - loss: 0.5305 - accuracy: 0.7755 - val_loss: 0.4346 - val_accuracy: 0.8223
Epoch 4/32
47494/47494 [==============================] - 222s 5ms/step - loss: 0.3840 - accuracy: 0.8457 - val_loss: 0.4415 - val_accuracy: 0.8186
Epoch 5/32
47494/47494 [==============================] - 224s 5ms/step - loss: 0.2797 - accuracy: 0.8949 - val_loss: 0.2908 - val_accuracy: 0.8891
Epoch 6/32
47494/47494 [==============================] - 226s 5ms/step - loss: 0.1981 - accuracy: 0.9280 - val_loss: 0.3600 - val_accuracy: 0.8680
Epoch 7/32
47494/47494 [===========================

In [ ]:
# remove train data from the ram
del(x_train)
del(y_train)

In [ ]:
# Load Test Data
x_test = np.load("x_test.npy", allow_pickle=True)
y_test = np.load("y_test.npy", allow_pickle=True)

In [ ]:
x_test = x_test.reshape(x_test.shape[0], 224, 224, 1)

In [ ]:
# Test model
result = model.evaluate(x_test, y_test)
print(result)

3294/3294 [==============================] - 4s 1ms/step
[0.5160460334314168, 0.8825136423110962]


In [ ]:
# remove test data from the ram
del(x_test)
del(y_test)

In [ ]:
ModelName= r"Trained-Final.h5"             # for saving
ModelSaveAddresss = r"/content/drive/My Drive/ML/Python Plays NFS/" + ModelName

In [ ]:
# Save the model
print("Saving...")           # for saving
model.save(ModelSaveAddresss.format(ModelName))              # save on google drive
model.save(ModelName)                                        # save on local storage
print("Done!")

Saving...
Done!


In [ ]:
#exit(0)